# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [ ]:
from openai import OpenAI
client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-tdqfro61NniG2LZd5394E5F28c51419d8b4dE7Db5a6105C4")

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

In [ ]:
response.choices[0].message.content

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默风格的搞笑比较。图中展示了两种不同状态的狗。左边是一只被幻想成拥有非常发达肌肉、类似人类健美运动员身体的柴犬，配文“16岁的我，工作后的我”，表达了年轻时充满活力和梦想的状态。右边的柴犬则显得普通、略显憔悴，配有文字“好累好困、好想摸鱼、看到鸡腿就开心，不属于我、我也有小肚腩、肉肉的很可爱”，描绘了工作后可能感到疲惫、减少锻炼而略显发福的现实状态。

这种对比通常用来幽默地表达人们对于理想与现实之间差距的感慨，展示了许多人对于年轻时的憧


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [9]:
from openai import OpenAI
import base64
import requests

client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-tdqfro61NniG2LZd5394E5F28c51419d8b4dE7Db5a6105C4")  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [10]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图表展示了1980年到2020年期间美国、中国、日本和德国的国内生产总值（GDP）对比。横坐标表示时间（年份），纵坐标表示GDP总量，单位为万亿美元。

从图中可以看出以下几点：
1. **美国（蓝色线）**的GDP在这40年间持续增长，增长幅度较为稳定，到2020年接近20万亿美元。
2. **中国（红色线）**的GDP从一个较低的起点开始，增长速度非常快，尤其是从2000年后，其增长速度显著加快，到2020年接近15万亿美元，紧追美国。
3. **日本（紫色线）**的GDP增长在1995年左右达到顶峰，此后增长放缓，到2000年后基本保持平稳，2020年GDP约为5万亿美元。
4. **德国（绿色线）**的GDP增长相对平稳，到2020年约为4万亿美元，与日本相比增长较慢。

这张图有效地展示了四个经济体在过去几十年的经济表现，特别是中国的快速崛起和日本的经济停滞。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [11]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片展示的是一本笔记本上手写的内容，主要关于自然语言处理和机器学习中的“Prompt Tuning”和“LoRA”的方法。笔记中记录了这两种技术的概念和一些相关的数学表达式。

1. **Prompt Tuning (适用于小模型)** 
   - 笔记提到了Prompt Tuning是适用于轻量化模型的调整方法。在调整过程中一般不改变原有的模型参数，而是通过添加或修改少量的“prompt”（提示）来引导模型输出所需的结果。
   - 笔记中还提到了如何构造输入X，X是由n个向量`X1, X2, ..., Xn`组成，这些向量是词嵌入后的向量化表示。输出Y是通过矩阵W与输入X的映射得到。

2. **LoRA (低秩适应技术)**
   - LoRA (Low-Rank Adaptation) 是另一种模型调整方法，主要目的是在不显著增加计算成本的情况下，通过添加低秩的矩阵来调整原有参数。
   - 表达式 `ΔW = AB` 中，AB表示一个低秩矩阵，其秩较低以减少参数量和计算复杂度。
   - 笔记中还讨论了这种方法在存储和计算上的优势，如通过Q_LoRA可以显著减少所需的存储空间。

这些笔记可能是某个正在学习或研究自然语言处理技术的学生或研究人员的记录，涉及到深度学习模型调整的高级技术。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [11]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一本笔记本上手写的文字，内容主要涉及到自然语言处理（NLP）中的训练技术，如Prompt Tuning和LoRA技术。

1. **Prompt Tuning（简要模型调整）**： 提到了使用Prompt Tuning来调整一个小型的Transformer模型。此处解释了输入X（由个别输入X1, X2, ..., Xn组成）。每个输入首先通过一个Embedding过程转换，然后通过Token变换。输出Y是通过矩阵W与转换后的输入X'之间的乘法得出。

2. **Prefix Tuning：** 这部分说明了Prefix Tuning的过程，其中添加了前缀权重W_p到原始权重W_j中，得到新的权重W'用于生成输出Y。

3. **LoRA调整技术**： 这部分涉及Linear Re-parameterization（线性重新参数化），通过调整矩阵ΔW（通过两个矩阵A和B的乘积表示）来修改权重W。这是一种节省参数调整的方法，使原有模型的W变为W+ΔW，这里也涉及到了一些矩阵运算和优化策略。

其中还提到了两个案例分析的存储需求：“LLAMA”需要65GB，而经过LoRA调整的“QLLoRA”仅需要48GB。

这些笔记对于理解NLP中一些先进的模型调整技术十分有用，尤其对于需要在资源受限的环境下部署NLP模型的研究人员或实践者。

![](./images/handwriting_1.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [8]:
from openai import OpenAI
import base64
import requests

client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-tdqfro61NniG2LZd5394E5F28c51419d8b4dE7Db5a6105C4")  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释图里是什么？并将其中内容翻译为中文", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

In [9]:
content = query_base64_image_description("./images/gpt-4v.jpg")
print(content)

这幅图示展示了一只手正在用笔在一本笔记本上写字。图片的风格是简约的线条绘图，使得注意力集中在写作动作上。

图中的日文文字是关于MD Notebook的广告文字。上面写着的是：

"言葉はいじわるには理由があります。" 
翻译为中文是：
"语言通常有其残酷的理由。"

下方的文字提到了MD Paper的一些背景信息，大致内容是：
“我们所使用的MD纸自1960年代以来被用作日记和笔记，其质感和耐久性使其获得了高度评价。计切纸张、无酸处理并采用易于书写的墨水，希望您能享受写字的乐趣。”

这幅图和文字一起表达了写作的感觉和这种特殊笔记本纸的特点。


In [10]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这幅图示展示了一只手正在用笔在一本笔记本上写字。图片的风格是简约的线条绘图，使得注意力集中在写作动作上。

图中的日文文字是关于MD Notebook的广告文字。上面写着的是：

"言葉はいじわるには理由があります。" 
翻译为中文是：
"语言通常有其残酷的理由。"

下方的文字提到了MD Paper的一些背景信息，大致内容是：
“我们所使用的MD纸自1960年代以来被用作日记和笔记，其质感和耐久性使其获得了高度评价。计切纸张、无酸处理并采用易于书写的墨水，希望您能享受写字的乐趣。”

这幅图和文字一起表达了写作的感觉和这种特殊笔记本纸的特点。